In [1]:
import pandas as pd
import numpy as np
import string

In [208]:
df = pd.read_csv('wip_p_l.csv')
df

,Unnamed: 0,Unnamed: 1,For the period ended 31 March 2020,For the year ended 30 June 2019
0,NaN,Note,BDT,BDT
1,Revenue,15,"79,64,17,162","1,35,64,93,099"
2,Cost of revenue,16,"-67,40,26,365","-1,04,42,38,749"
3,Gross profit,NaN,"12,23,90,797","31,22,54,350"
4,Administrative expenses,17,"-8,15,76,422","-5,18,53,744"
5,Profit from operations,NaN,"4,08,14,375","26,04,00,606"
6,Other income,NaN,"2,74,65,369","46,03,855"
7,Profit before tax,NaN,"6,82,79,743","26,50,04,461"
8,Income tax expense,18,-,"10,21,85,214"
9,Net profit for the period,NaN,"6,82,79,743","16,28,19,247"


In [242]:
df= df.rename(columns={"Unnamed: 1":"Notes", "For the year ended 30 June 2019 ":"30 June 2019", "For the period ended 31 March 2020 ":"31 March 2020"}, errors="raise")


TypeError: 'Series' object is not callable

In [243]:
new_header = df.iloc[0] #grab the first row for the header
df.columns.values[0] = 'Names'
df.columns.drop(list(df.filter(regex='Note')))
df.dropna(how='all', axis=1, inplace=True)
df = df.loc[:, df.columns.notnull()]
df


,Names,Notes,31 March 2020,30 June 2019
0,NaN,Note,BDT,BDT
1,Revenue,15,"79,64,17,162","1,35,64,93,099"
2,Cost of revenue,16,"-67,40,26,365","-1,04,42,38,749"
3,Gross profit,NaN,"12,23,90,797","31,22,54,350"
4,Administrative expenses,17,"-8,15,76,422","-5,18,53,744"
5,Profit from operations,NaN,"4,08,14,375","26,04,00,606"
6,Other income,NaN,"2,74,65,369","46,03,855"
7,Profit before tax,NaN,"6,82,79,743","26,50,04,461"
8,Income tax expense,18,-,"10,21,85,214"
9,Net profit for the period,NaN,"6,82,79,743","16,28,19,247"


In [244]:
df1 = df.replace(np.nan, '0', regex=True)
df1

,Names,Notes,31 March 2020,30 June 2019
0,0,Note,BDT,BDT
1,Revenue,15,"79,64,17,162","1,35,64,93,099"
2,Cost of revenue,16,"-67,40,26,365","-1,04,42,38,749"
3,Gross profit,0,"12,23,90,797","31,22,54,350"
4,Administrative expenses,17,"-8,15,76,422","-5,18,53,744"
5,Profit from operations,0,"4,08,14,375","26,04,00,606"
6,Other income,0,"2,74,65,369","46,03,855"
7,Profit before tax,0,"6,82,79,743","26,50,04,461"
8,Income tax expense,18,-,"10,21,85,214"
9,Net profit for the period,0,"6,82,79,743","16,28,19,247"


In [265]:
TotalcolumnArray = list(df1.columns)
TotalcolumnArray.pop(0)
print(TotalcolumnArray)
for i in TotalcolumnArray:
    df1[i] = df1[i].astype(str)
    df1[i] = df1[i].apply(lambda x:''.join([i.replace('$','') for i in list(x)]))
    df1[i] = df1[i].apply(lambda x:''.join([i.replace('(','-').replace(',','').replace(')','') for i in list(x)]))
    df1[[i]] = df1[[i]].apply(pd.to_numeric, errors='coerce')

    
df1 = df1.replace(np.nan, '0.0', regex=True)
df1

['Notes', '31 March 2020', '30 June 2019', 'Type']


,Names,Notes,31 March 2020,30 June 2019,Type
0,0,0.0,0.0,0.000000e+00,0.0
1,Revenue,15.0,796417162.0,1.356493e+09,0.0
2,Cost of revenue,16.0,-674026365.0,-1.044239e+09,0.0
3,Gross profit,0.0,122390797.0,3.122544e+08,0.0
4,Administrative expenses,17.0,-81576422.0,-5.185374e+07,0.0
5,Profit from operations,0.0,40814375.0,2.604006e+08,0.0
6,Other income,0.0,27465369.0,4.603855e+06,0.0
7,Profit before tax,0.0,68279743.0,2.650045e+08,0.0
8,Income tax expense,18.0,0.0,1.021852e+08,0.0
9,Net profit for the period,0.0,68279743.0,1.628192e+08,0.0


In [266]:
conditions = [
            (df1['Names'].str.contains("^Revenue", na=False, case=False)),#REVENUE
            (df1['Names'].str.contains("Cost of", na=False, case=False)) |
            (df1['Names'].str.contains("COGS", na=False, case=False)), #COGS
            (df1['Names'].str.contains("Administrative expenses", na=False, case=False)) | 
            (df1['Names'].str.contains("General Expenses", na=False, case=False)) | 
            (df1['Names'].str.contains("Salary", na=False, case=False)) | 
            (df1['Names'].str.contains("Training", na=False, case=False)) | 
            (df1['Names'].str.contains("welfare fund", na=False, case=False)) | 
            (df1['Names'].str.contains("Repair and Maintenance", na=False, case=False)) | 
            (df1['Names'].str.contains("Electricity", na=False, case=False)) |
            (df1['Names'].str.contains("Rents and Rates", na=False, case=False)) |
            (df1['Names'].str.contains("Technology", na=False, case=False)) |
            (df1['Names'].str.contains("other office costs", na=False, case=False)) |
            (df1['Names'].str.contains("Legal Fees", na=False, case=False)) | 
            (df1['Names'].str.contains("Advisory Fees", na=False, case=False)) | 
            (df1['Names'].str.contains("Accounting and Audit Fees", na=False, case=False)) |
            (df1['Names'].str.contains("Other operating (cost|expense)", na=False, case=False)), #ADMINISTRATIVE EXP
            (df1['Names'].str.contains("depreciation adjustments", na=False, case=False)) |
            (df1['Names'].str.contains("Depreciation - Fixed assets", na=False, case=False)) | 
            (df1['Names'].str.contains("Operating & Lease amortisation", na=False, case=False)), #DEPRI_AMORT_TION
            (df1['Names'].str.contains("Selling cost", na=False, case=False)) | 
            (df1['Names'].str.contains("Promotional Expenses", na=False, case=False)) | 
            (df1['Names'].str.contains("Delivery Charges", na=False, case=False)) | 
            (df1['Names'].str.contains("Advertising", na=False, case=False)) |
            (df1['Names'].str.contains("Marketing Costs", na=False, case=False)) |
            (df1['Names'].str.contains("Social Media", na=False, case=False)) |
            (df1['Names'].str.contains("Events", na=False, case=False)) |
            (df1['Names'].str.contains("Seminars", na=False, case=False)) ,#SALES_MARKETING
            (df1['Names'].str.contains("HP interests", na=False, case=False)) |
            (df1['Names'].str.contains("Bank interests", na=False, case=False)) |
            (df1['Names'].str.contains("bank loans", na=False, case=False)) |
            (df1['Names'].str.contains("Exchange (Gain)/ Loss", na=False, case=False)) ,#FINANCIAL
            (df1['Names'].str.contains("Property tax", na=False, case=False)) , #PROPERTY_TAX
            (df1['Names'].str.contains("(Sample )*Other (Operating )*Income", na=False, case=False)),#OTH_OP_INC 
            (df1['Names'].str.contains("Investment income", na=False, case=False)) | 
            (df1['Names'].str.contains("Other Non-operating Income", na=False, case=False)),#OTH_NON_OP_INC            (df1['Names'].str.contains("Other Non Operating Expenses", na=False, case=False)) ,
            (df1['Names'].str.contains("Other Non-operating Expense", na=False, case=False)),  #OTH_NON_OP_EXP
            (df1['Names'].str.contains("Income tax", na=False, case=False))  | 
            (df1['Names'].str.contains("Deferred Tax", na=False, case=False)) , #INCOME_TAX
            (df1['Names'].str.contains("Dividends", na=False, case=False)) ,#DIVIDENDS
            (df1['Names'].str.contains("Other Income/Expense Net of Tax", na=False, case=False)) |
            (df1['Names'].str.contains("Other comprehensive income", na=False, case=False))   #OTH_INC_EXP_NET_OF_TAX 
        ]

# create a list of the values we want to assign for each condition
values = ['REVENUE', 'COGS', 'ADMINISTRATIVE_EXP', 'DEPRI_AMORT_TION', 'SALES_MARKETING','FINANCIAL', 'PROPERTY_TAX', 
             'OTH_OP_INC', 'OTH_NON_OP_INC', 'OTH_NON_OP_EXP', 'INCOME_TAX', 'DIVIDENDS', 'OTH_INC_EXP_NET_OF_TAX' ]

# create a new column and use np.select to assign values to it using our lists as arguments
df1['Type'] = np.select(conditions, values)

df1

/Users/debasishaddy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Names,Notes,31 March 2020,30 June 2019,Type
0,0,0.0,0.0,0.000000e+00,0
1,Revenue,15.0,796417162.0,1.356493e+09,REVENUE
2,Cost of revenue,16.0,-674026365.0,-1.044239e+09,COGS
3,Gross profit,0.0,122390797.0,3.122544e+08,0
4,Administrative expenses,17.0,-81576422.0,-5.185374e+07,ADMINISTRATIVE_EXP
5,Profit from operations,0.0,40814375.0,2.604006e+08,0
6,Other income,0.0,27465369.0,4.603855e+06,OTH_OP_INC
7,Profit before tax,0.0,68279743.0,2.650045e+08,0
8,Income tax expense,18.0,0.0,1.021852e+08,INCOME_TAX
9,Net profit for the period,0.0,68279743.0,1.628192e+08,0


In [267]:
# display updated DataFrame
df2=df1.groupby('Type').agg({'31 March 2020' : 'sum','30 June 2019' : 'sum'}, errors='raise')
df2

,31 March 2020,30 June 2019
Type,,
0,368044401.0,1.163298e+09
ADMINISTRATIVE_EXP,-81576422.0,-5.185374e+07
COGS,-674026365.0,-1.044239e+09
INCOME_TAX,0.0,1.021852e+08
OTH_INC_EXP_NET_OF_TAX,0.0,0.000000e+00
OTH_OP_INC,27465369.0,4.603855e+06
REVENUE,796417162.0,1.356493e+09


In [226]:
df_p_l_out = pd.read_csv('Master File for Financial Ratios -July 21.csv')
#df_p_l_out.iloc(6:2, axis=1)=

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,PROFIT AND LOSS STATEMENT,COMPANY NAME,NaN,NaN,NaN,NaN
1,NaN,Transposed from Submitted Financial Statement,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Currency,Audited,Audited,Comments,Checks,Source Document
4,NaN,NaN,2018,2017,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,I,Revenue,NaN,NaN,OVERALL GUIDELINE - PLEASE DO NOT CHANGE THIS ...,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,(a),Revenue from Operations,NaN,NaN,Gross Sales + Other Operating Revenues and income,NaN,NaN
9,(b),Other income,NaN,NaN,Interst income + Dividends income + Govt incen...,NaN,NaN


In [184]:
df_profit_loss.to_csv('wip_p_l_op.csv')